<a href="https://colab.research.google.com/github/karailker/colab_projects/blob/main/FAISS_NNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FAISS - NearestNeighbourSearch

In [1]:
!pip list | grep "faiss"

In [3]:
%pip install -U faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.4 MB/s 


In [19]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 4.4 MB 37.7 MB/s 
     |████████████████████████████████| 1.2 MB 34.8 MB/s 
     |████████████████████████████████| 101 kB 9.8 MB/s 
     |████████████████████████████████| 596 kB 38.4 MB/s 
     |████████████████████████████████| 6.6 MB 27.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=efcf4ae744ed99da571f2ac8bcc7225dacee98f505d85cb6aec1738f384f7e7e
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
!apt install libomp-dev -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (281 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Se

In [8]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

## Sentence Embeddings

In [9]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

In [10]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [11]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [12]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

In [13]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [14]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'


In [15]:
len(set(sentences))

14505

In [16]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [17]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

In [20]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

(14504, 768)

In [23]:
sentence_embeddings.shape[0]

14504

In [24]:
with open(f'embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [25]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

In [26]:
sentence_embeddings[0].shape

(768,)

In [27]:
d = sentence_embeddings.shape[1]
d

768

## FAISS

In [28]:
index = faiss.IndexFlatL2(d)

In [29]:
index.is_trained

True

In [30]:
index.ntotal

0

In [31]:
index.add(sentence_embeddings)

In [32]:
index.ntotal

14504

In [33]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [34]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495

[[ 6249 13901  3101  3326]]
CPU times: user 20.7 ms, sys: 4.99 ms, total: 25.7 ms
Wall time: 26.2 ms


In [35]:
[f'{i}: {sentences[i]}' for i in I[0]]

['6249: A group of football players is running in the field',
 '13901: A group of people playing football is running in the field',
 '3101: Two groups of people are playing football',
 '3326: A person playing football is running past an official carrying a football']

In [36]:
sentences[7460]

'A child is falling asleep'

In [37]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [38]:
vecs.shape

(4, 768)

In [39]:
vecs[0][:100]

array([ 0.01627023,  0.22325914, -0.15037395, -0.30747256, -0.27122432,
       -0.10593218, -0.06460925,  0.04738221, -0.73349053, -0.37657711,
       -0.76762813,  0.16902894,  0.53107667,  0.51176625,  1.14415848,
       -0.08562878, -0.67240089, -0.96637082,  0.02545462, -0.21559827,
       -1.25656617, -0.82982183, -0.09824976, -0.21850877,  0.50610268,
        0.10527939,  0.50396883,  0.65242964, -1.39458692,  0.65847474,
       -0.21525346, -0.22487427,  0.81818342,  0.08464307, -0.76141673,
       -0.2892831 , -0.09825789, -0.73046154,  0.07855817, -0.84354573,
       -0.59242088,  0.7747137 , -1.20920563, -0.22757973, -1.30733597,
       -0.23081492, -1.31322515,  0.01629092, -0.97285461,  0.19308168,
        0.47424552,  1.1892091 , -1.96741259, -0.70061117, -0.29638746,
        0.60533696,  0.62407428, -0.70340377, -0.86754185,  0.17673105,
       -0.19170566, -0.02951992,  0.22623523, -0.16695441, -0.80402523,
       -0.4591895 ,  0.69675511, -0.24928199, -1.01478696, -0.92

###Index Partitioning

In [40]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [41]:
index.is_trained

False

In [42]:
index.train(sentence_embeddings)
index.is_trained

True

In [43]:
index.add(sentence_embeddings)
index.ntotal

14504

In [44]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 6249 13901  3326  3407]]
CPU times: user 5.68 ms, sys: 0 ns, total: 5.68 ms
Wall time: 5.81 ms


In [45]:
index.nprobe = 10

In [47]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 6249 13901  3101  3326]]
CPU times: user 11.1 ms, sys: 12 µs, total: 11.2 ms
Wall time: 12.3 ms


In [49]:
index.make_direct_map()

In [50]:
index.reconstruct(7460)[:100]

array([-1.5619032e-01, -7.9146281e-02,  1.3769133e+00, -1.9398297e-01,
       -4.5425436e-01,  5.1871204e-01,  6.4227277e-01,  1.4270753e-01,
       -8.0248117e-03, -1.4048666e-01, -9.0029395e-01, -9.2322692e-02,
        9.1603082e-01,  7.6279050e-01,  6.2934235e-02, -5.1949996e-01,
        2.0347527e-01, -6.1794394e-01,  7.1519405e-01, -3.3116189e-01,
       -3.7475869e-01,  3.5305882e-01, -8.1299841e-01, -1.0981410e+00,
       -4.8327658e-01, -7.5469774e-01, -7.9252356e-01, -1.0608603e-01,
       -1.3805608e+00, -5.2437979e-01, -1.5196033e-01,  1.4884738e+00,
        1.0447968e+00,  4.6972513e-01,  1.0286134e+00, -2.4399140e-01,
        1.0760776e+00, -1.4712188e+00, -1.1101450e-01, -3.9612648e-01,
        4.6640828e-01,  1.6520520e-03, -2.3620164e-01, -4.3698192e-01,
        9.6251047e-01, -7.2237635e-01,  1.2405804e+00,  1.9315818e-01,
        5.8386678e-01, -2.0197318e+00, -5.6229442e-02, -1.3895817e-01,
        2.3567748e+00,  4.0760687e-01, -5.9088147e-01, -8.9601642e-01,
      

### Quantization

In [51]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [52]:
index.is_trained

False

In [53]:
index.train(sentence_embeddings)

In [54]:
index.add(sentence_embeddings)

In [55]:
index.nprobe = 10

In [56]:
%%time
D, I = index.search(xq, k)
print(I)

[[3326 6249 3407 3235]]
CPU times: user 3.82 ms, sys: 1.01 ms, total: 4.84 ms
Wall time: 4.1 ms


In [60]:
[f'{i}: {sentences[i]}' for i in I[0]]

['3326: A person playing football is running past an official carrying a football',
 '6249: A group of football players is running in the field',
 '3407: A group of football players running down the field.',
 '3235: A football player is running past an official carrying a football']